In [1]:
import os
from pathlib import Path
import time
import yaml
from typing import Dict, List, Union

In [2]:
def generate_envoy_configs(config: Dict,
                           save_path: Union[str, Path] = '../envoy/',
                           n_cols_total: int = 10,
                           n_cols_corrupt: int = 1,
                           assign_gpus: bool = False,
                           image_hw: str = '300,400') -> List[Path]:
    
    config['shard_descriptor']['params']['enforce_image_hw'] = image_hw
    
    config_paths = [(Path(save_path) / f'{i}_envoy_config.yaml').absolute()
                for i in range(1, n_cols_total + 1)]

    for i, path in enumerate(config_paths):
        if assign_gpus:
            config['params']['cuda_devices'] = [i,]
        else:
            config['params']['cuda_devices'] = []
        if i < n_cols_corrupt:
            config['shard_descriptor']['params']['corrupt'] = True
        else:
            config['shard_descriptor']['params']['corrupt'] = False
        config['shard_descriptor']['params']['rank_worldsize'] = f'{i+1},{n_cols_total}'
        with open(path, "w") as stream:
            yaml.safe_dump(config, stream)
            
    return config_paths
            
def remove_configs(config_paths):
    for path in config_paths:
        path.unlink()

## Start the Director service

In [3]:
# cwd = Path.cwd()
# director_workspace_path = Path('../director/').absolute()
# director_config_file = director_workspace_path / 'director_config.yaml'
# director_logfile = director_workspace_path / 'director.log'
# director_logfile.unlink(missing_ok=True)
# # 

# os.environ['main_folder'] = str(cwd)
# os.environ['director_workspace_path'] = str(director_workspace_path)
# os.environ['director_logfile'] = str(director_logfile)
# os.environ['director_config_file'] = str(director_config_file)

In [4]:
# %%script /bin/bash --bg
# cd $director_workspace_path
# fx director start --disable-tls -c $director_config_file > $director_logfile &
# cd $main_folder

In [5]:
def start_director():
    cwd = Path.cwd()
    director_workspace_path = Path('../director/').absolute()
    os.chdir(director_workspace_path)
    director_config_file = director_workspace_path / 'director_config.yaml'
    director_logfile = director_workspace_path / 'director.log'
    director_logfile.unlink(missing_ok=True)
    os.system('fx director start --disable-tls '
              f'-c {director_config_file} > {director_logfile} &')
    os.chdir(cwd)

## Start Envoys

In [6]:
# envoy_workspace_path = Path('../envoy/').absolute()
"""Make sure the Director port matches one from the Director config file"""
def start_envoys(config_paths: List[Path]) -> None:
    envoy_workspace_path = config_paths[0].parent
    cwd = Path.cwd()
    os.chdir(envoy_workspace_path)
    for i, path in enumerate(config_paths):
        os.system(f'fx envoy start -n env_{i + 1} --disable-tls '
                  f'--envoy-config-path {path} -dh localhost -dp 50050 '
                  f'>env_{i + 1}.log &')
    os.chdir(cwd)
    
# start_envoys(config_paths)

In [7]:
def start_federation(envoy_config_path='../envoy/envoy_config.yaml',
                    n_cols_total=3, n_cols_corrupt=0, deacticate_cols_indeces=(),
                    assign_gpus=True):
    # Read the original envoy config file content
    with open(Path(envoy_config_path), "r") as stream:
        orig_config = yaml.safe_load(stream)
    # Write new configs
    config_paths = generate_envoy_configs(orig_config, n_cols_total=n_cols_total,
                                          n_cols_corrupt=n_cols_corrupt, assign_gpus=assign_gpus)
    for idx in deacticate_cols_indeces:
        del config_paths[idx]
    # Start Director and Envoys 
    start_director()
    time.sleep(2)
    start_envoys(config_paths)
    time.sleep(2)
#     remove_configs(config_paths)

## Run experiments

In [8]:
# from openfl.component.aggregation_functions import AggregationFunction
# import numpy as np
# class One_Good_Envoy(AggregationFunction):
#     def __init__(self, col_name='env_3', weight_scale: float = 0.5):
#         self.good_col = col_name
#         self.weight_scale = weight_scale

#     def call(self, local_tensors, *_) -> np.ndarray:
#         weights = [x.weight if self.good_col == x.col_name else x.weight * self.weight_scale
#                    for x in local_tensors]
#         tensors = np.array([x.tensor for x in local_tensors])
#         return np.average(tensors, weights=weights, axis=0)

In [9]:
from testbook import testbook


def start_experiment_from_ipynb(experiment_name, notebook_path='./PyTorch_Kvasir_UNet.ipynb',
                                n_rounds=40, train_batch_size=6, lr=1e-4, weight_decay=0,
                               opt_treatment='CONTINUE_GLOBAL'):
    command = (
         'torch.manual_seed(0) \n'
         'model_unet = UNet() \n'
        f'optimizer_adam = optim.Adam(model_unet.parameters(), lr={lr}, weight_decay={weight_decay}) \n'
         'MI = ModelInterface(model=model_unet, optimizer=optimizer_adam, framework_plugin=framework_adapter) \n'
        f'fed_dataset = KvasirSD(train_bs={train_batch_size}, valid_bs=8) \n'
        f'fl_experiment = FLExperiment(federation=federation, experiment_name="{experiment_name}") \n'
         'fl_experiment.start(model_provider=MI, task_keeper=TI, data_loader=fed_dataset, \n'
                       f'rounds_to_train={n_rounds}, \n'
                       f"opt_treatment='{opt_treatment}', \n"
                        "device_assignment_policy='CUDA_PREFERRED') \n"
         'fl_experiment.stream_metrics()'
    )
    # We may execute only some cells, i.e. range(0,23)
    # We should not execute `start_experiment`
    with testbook(notebook_path, execute=True, timeout=None) as tb:
        tb.inject(command, pop=True)

## Stop the Federation and Clean up

In [18]:
# To stop all services run
# !pkill fx
os.system('pkill fx')

256

# Run all together 

In [19]:
n_cols_total=3
n_cols_corrupt=0
n_rounds=50
train_batch_size=6
lr=1.5e-4
weight_decay=1e-6
for n_cols_corrupt in [2]:
    try:
        start_federation(n_cols_total=n_cols_total, n_cols_corrupt=n_cols_corrupt,
                        deacticate_cols_indeces=(0,1))
        experiment_name = (
            f'KVSR_rounds{n_rounds}_bs{train_batch_size}_'
            f'off{n_cols_corrupt}_of{n_cols_total}'
        )
        start_experiment_from_ipynb(experiment_name=experiment_name,
                                n_rounds=n_rounds, train_batch_size=train_batch_size,
                               lr=lr, weight_decay=weight_decay)
    finally:
        os.system('pkill fx')
    

File ‘kvasir_data/kvasir.zip’ already there; not retrieving.
You should consider upgrading via the '/home/idavidyu/.virtualenvs/corrupt-envoy/bin/python -m pip install --upgrade pip' command.
/home/idavidyu/openfl/openfl/plugins/frameworks_adapters/pytorch_adapter.py:47: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  new_state[k] = pt.from_numpy(tensor_dict.pop(k)).to(device)
validate:   0%|          | 0/6 [00:00<?, ?it/s]/tmp/ipykernel_594119/278825291.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone()

validate:   0%|          | 0/6 [00:00<?, ?it/s]/tmp/ipykernel_594119/278825291.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/tmp/ipykernel_594119/278825291.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
validate:   0%|          | 0/6 [00:00<?, ?it/s]/tmp/ipykernel_594119/278825291.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/tmp/ipykernel_594119/278825291.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather

validate:   0%|          | 0/6 [00:00<?, ?it/s]/tmp/ipykernel_594119/278825291.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/tmp/ipykernel_594119/278825291.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
validate:   0%|          | 0/6 [00:00<?, ?it/s]/tmp/ipykernel_594119/278825291.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/tmp/ipykernel_594119/278825291.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather

validate:   0%|          | 0/6 [00:00<?, ?it/s]/tmp/ipykernel_594119/278825291.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/tmp/ipykernel_594119/278825291.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
validate:   0%|          | 0/6 [00:00<?, ?it/s]/tmp/ipykernel_594119/278825291.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/tmp/ipykernel_594119/278825291.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather

validate:   0%|          | 0/6 [00:00<?, ?it/s]/tmp/ipykernel_594119/278825291.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/tmp/ipykernel_594119/278825291.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
validate:   0%|          | 0/6 [00:00<?, ?it/s]/tmp/ipykernel_594119/278825291.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/tmp/ipykernel_594119/278825291.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather

validate:   0%|          | 0/6 [00:00<?, ?it/s]/tmp/ipykernel_594119/278825291.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/tmp/ipykernel_594119/278825291.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
validate:   0%|          | 0/6 [00:00<?, ?it/s]/tmp/ipykernel_594119/278825291.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/tmp/ipykernel_594119/278825291.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather

validate:   0%|          | 0/6 [00:00<?, ?it/s]/tmp/ipykernel_594119/278825291.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/tmp/ipykernel_594119/278825291.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
validate:   0%|          | 0/6 [00:00<?, ?it/s]/tmp/ipykernel_594119/278825291.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/tmp/ipykernel_594119/278825291.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather

validate:   0%|          | 0/6 [00:00<?, ?it/s]/tmp/ipykernel_594119/278825291.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/tmp/ipykernel_594119/278825291.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
validate:   0%|          | 0/6 [00:00<?, ?it/s]/tmp/ipykernel_594119/278825291.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/tmp/ipykernel_594119/278825291.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather

validate:   0%|          | 0/6 [00:00<?, ?it/s]/tmp/ipykernel_594119/278825291.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/tmp/ipykernel_594119/278825291.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
validate:   0%|          | 0/6 [00:00<?, ?it/s]/tmp/ipykernel_594119/278825291.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/tmp/ipykernel_594119/278825291.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather

In [12]:
[1,2,3,4][0]

1